In [1]:
from google.colab import files
files.upload()

Saving analisis_comments_tiktok.csv to analisis_comments_tiktok.csv


{'analisis_comments_tiktok.csv': b'date_post,date_extraction,influencer,post_type,post_url,platform,comment,comment_likecount,replies,BU;;;;;;;;;\r\n"2021-9-12,2021-12-01,espdaniella,VIDEO,https://www.tiktok.com/@espdaniella/video/7007109992926989574?lang=en,TikTok,""Oh so it\xe2\x80\x99s an ad lol, how do I know if it\xe2\x80\x99s even good then \xf0\x9f\x92\x80"",27,,CS";;;;;;;;;\r\n2021-9-12,2021-12-01,espdaniella,VIDEO,https://www.tiktok.com/@espdaniella/video/7007109992926989574?lang=en,TikTok,it\xe2\x80\x99s not good chegg apparently rats out on students,14,,CS;;;;;;;;;\r\n"2021-9-11,2021-12-01,themccartys,VIDEO,https://www.tiktok.com/@themccartys/video/7006777814015855878?lang=en,TikTok,""Wow is she single? She is pretty cute, especially with that haircut. \xf0\x9f\xa5\xb0\xf0\x9f\x98\x82"",10,,CS";;;;;;;;;\r\n2021-9-11,2021-12-01,themccartys,VIDEO,https://www.tiktok.com/@themccartys/video/7006777814015855878?lang=en,TikTok,I hope Audri looks different at that age,71,,CS;;;;;;;;

In [2]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


vADER’s SentimentIntensityAnalyzer() takes in a string and returns a dictionary of scores in each of four categories:
negative
neutral
positive
compound (computed by normalizing the scores above

In [4]:
a = 'This was a good movie.'
sid.polarity_scores(a)

a = 'This was the best, most awesome movie EVER MADE!!!'
sid.polarity_scores(a)


{'compound': 0.8877, 'neg': 0.0, 'neu': 0.425, 'pos': 0.575}

In [5]:
import numpy as np
import pandas as pd
df = pd.read_csv('analisis_comments_tiktok.csv')
df.head()

,date_post,date_extraction,influencer,post_type,post_url,platform,comment,comment_likecount,replies,BU;;;;;;;;;
0,"2021-9-12,2021-12-01,espdaniella,VIDEO,https:/...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-9-12,2021-12-01,espdaniella,VIDEO,https://www.tiktok.com/@espdaniella/video/7007...,TikTok,it’s not good chegg apparently rats out on stu...,14,NaN,CS;;;;;;;;;
2,"2021-9-11,2021-12-01,themccartys,VIDEO,https:/...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-9-11,2021-12-01,themccartys,VIDEO,https://www.tiktok.com/@themccartys/video/7006...,TikTok,I hope Audri looks different at that age,71,NaN,CS;;;;;;;;;
4,2021-9-11,2021-12-01,themccartys,VIDEO,https://www.tiktok.com/@themccartys/video/7006...,TikTok,I absolutely love it!,26,NaN,CS;;;;;;;;;


In [8]:
df['scores'] = df['comment'].apply(lambda review: sid.polarity_scores(review))

df.head()

,date_post,date_extraction,influencer,post_type,post_url,platform,comment,comment_likecount,replies,BU;;;;;;;;;,scores
41,2021-10-14,2021-12-15,JJ Medicine,VIDEO,https://www.youtube.com/watch?v=FwiaoKrdzx4,Youtube,I&#39;ve heard this often goes with anciety/ d...,0,I too had mono...ever since high Epstein bar l...,CS;;;;;,"{'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'comp..."
51,2021-10-14,2021-12-15,JJ Medicine,VIDEO,https://www.youtube.com/watch?v=FwiaoKrdzx4,Youtube,Falling asleep whilst watching this....zzzzzzzzx,0,Coffee 😁😁,CS;;;;;;;;;,"{'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'comp..."
56,2021-10-14,2021-12-15,JJ Medicine,VIDEO,https://www.youtube.com/watch?v=FwiaoKrdzx4,Youtube,Falling asleep whilst watching this....zzzzzzzzx,0,While?,CS;;;;;;;;;,"{'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'comp..."
89,2021-11-22,2021-12-15,ninjanerdscience,VIDEO,https://www.youtube.com/watch?v=UJw2WUtHLIk,Youtube,Ninja nerd I&#39;m so happy that you choose th...,0,Ulcerative Colitis is next to be released! Hop...,CS;;;;;;;;,"{'neg': 0.068, 'neu': 0.622, 'pos': 0.31, 'com..."
96,"2021-11-22,2021-12-15,ninjanerdscience,VIDEO,h...",ad many other diseases. What&#39;s the word ...,as usual,"it&#39;""s hard to distinguish cause from effe...",it seems that substantial D3 supplementation ...,0,That’s interesting. I’m a nurse on a GI ward a...,it maybe causation,"it may be just me looking out for it.""""","CS"";;;;;;","{'neg': 0.0, 'neu': 0.915, 'pos': 0.085, 'comp..."


In [9]:
df['compound']  = df['scores'].apply(lambda score_dict: score_dict['compound'])

df.head()

,date_post,date_extraction,influencer,post_type,post_url,platform,comment,comment_likecount,replies,BU;;;;;;;;;,scores,compound
41,2021-10-14,2021-12-15,JJ Medicine,VIDEO,https://www.youtube.com/watch?v=FwiaoKrdzx4,Youtube,I&#39;ve heard this often goes with anciety/ d...,0,I too had mono...ever since high Epstein bar l...,CS;;;;;,"{'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'comp...",-0.7783
51,2021-10-14,2021-12-15,JJ Medicine,VIDEO,https://www.youtube.com/watch?v=FwiaoKrdzx4,Youtube,Falling asleep whilst watching this....zzzzzzzzx,0,Coffee 😁😁,CS;;;;;;;;;,"{'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'comp...",-0.1531
56,2021-10-14,2021-12-15,JJ Medicine,VIDEO,https://www.youtube.com/watch?v=FwiaoKrdzx4,Youtube,Falling asleep whilst watching this....zzzzzzzzx,0,While?,CS;;;;;;;;;,"{'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'comp...",-0.1531
89,2021-11-22,2021-12-15,ninjanerdscience,VIDEO,https://www.youtube.com/watch?v=UJw2WUtHLIk,Youtube,Ninja nerd I&#39;m so happy that you choose th...,0,Ulcerative Colitis is next to be released! Hop...,CS;;;;;;;;,"{'neg': 0.068, 'neu': 0.622, 'pos': 0.31, 'com...",0.8303
96,"2021-11-22,2021-12-15,ninjanerdscience,VIDEO,h...",ad many other diseases. What&#39;s the word ...,as usual,"it&#39;""s hard to distinguish cause from effe...",it seems that substantial D3 supplementation ...,0,That’s interesting. I’m a nurse on a GI ward a...,it maybe causation,"it may be just me looking out for it.""""","CS"";;;;;;","{'neg': 0.0, 'neu': 0.915, 'pos': 0.085, 'comp...",0.4019


In [10]:
df['comp_score'] = df['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')

df.head()



,date_post,date_extraction,influencer,post_type,post_url,platform,comment,comment_likecount,replies,BU;;;;;;;;;,scores,compound,comp_score
41,2021-10-14,2021-12-15,JJ Medicine,VIDEO,https://www.youtube.com/watch?v=FwiaoKrdzx4,Youtube,I&#39;ve heard this often goes with anciety/ d...,0,I too had mono...ever since high Epstein bar l...,CS;;;;;,"{'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'comp...",-0.7783,neg
51,2021-10-14,2021-12-15,JJ Medicine,VIDEO,https://www.youtube.com/watch?v=FwiaoKrdzx4,Youtube,Falling asleep whilst watching this....zzzzzzzzx,0,Coffee 😁😁,CS;;;;;;;;;,"{'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'comp...",-0.1531,neg
56,2021-10-14,2021-12-15,JJ Medicine,VIDEO,https://www.youtube.com/watch?v=FwiaoKrdzx4,Youtube,Falling asleep whilst watching this....zzzzzzzzx,0,While?,CS;;;;;;;;;,"{'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'comp...",-0.1531,neg
89,2021-11-22,2021-12-15,ninjanerdscience,VIDEO,https://www.youtube.com/watch?v=UJw2WUtHLIk,Youtube,Ninja nerd I&#39;m so happy that you choose th...,0,Ulcerative Colitis is next to be released! Hop...,CS;;;;;;;;,"{'neg': 0.068, 'neu': 0.622, 'pos': 0.31, 'com...",0.8303,pos
96,"2021-11-22,2021-12-15,ninjanerdscience,VIDEO,h...",ad many other diseases. What&#39;s the word ...,as usual,"it&#39;""s hard to distinguish cause from effe...",it seems that substantial D3 supplementation ...,0,That’s interesting. I’m a nurse on a GI ward a...,it maybe causation,"it may be just me looking out for it.""""","CS"";;;;;;","{'neg': 0.0, 'neu': 0.915, 'pos': 0.085, 'comp...",0.4019,pos


In [11]:
# Write a review as one continuous string (multiple sentences are ok)
review = 'The shoes I brought were amazing.'
# Obtain the sid scores for your review
sid.polarity_scores(review)

{'neg': 0.0, 'neu': 0.513, 'pos': 0.487, 'compound': 0.5859}
review='The mobile phone I bought was the WORST and very BAD'
# Obtain the sid scores for your review
sid.polarity_scores(review)



{'compound': -0.8849, 'neg': 0.539, 'neu': 0.461, 'pos': 0.0}

In [14]:
pip install plotly

In [12]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))

In [16]:
pip install cufflinks

In [17]:
from plotly.offline import init_notebook_mode,iplot
import plotly.graph_objects as go
import cufflinks as cf
init_notebook_mode(connected=True)

In [18]:
#labels
lab = df["comp_score"].value_counts().keys().tolist()
#values
val = df["comp_score"].value_counts().values.tolist()
trace = go.Pie(labels=lab, 
                values=val, 
                marker=dict(colors=['red']), 
                # Seting values to 
                hoverinfo="value"
              )
data = [trace]

In [20]:
layout = go.Layout(title="Sentiment Distribution")
fig = go.Figure(data = data,layout = layout)
iplot(fig)